In [19]:
import torch
from datasets import load_dataset, DatasetDict, concatenate_datasets
from transformers import AutoTokenizer, Trainer, TrainingArguments, AutoModelForSequenceClassification, BertForSequenceClassification, DataCollatorWithPadding


import numpy as np
import matplotlib.pyplot as plt
import evaluate
import pandas as pd
import scipy
dataset = load_dataset("clinc_oos", "imbalanced").rename_column("intent", "labels")
# dataset = load_dataset("mtod", data_files=data_file_names)
dataset_full = concatenate_datasets([dataset["train"], dataset["test"], dataset['validation']]).shuffle(seed=42)
train_size, cal_size, test_size = .6, .2, .2
train_test_set = dataset_full.train_test_split(train_size=train_size, stratify_by_column='labels', seed=42)
test_val_set = train_test_set["test"].train_test_split(train_size=test_size/(cal_size+test_size), stratify_by_column='labels', seed=42)
dataset = DatasetDict({
    'train': train_test_set['train'],
    'test': test_val_set['train'],
    'validation': test_val_set['test']  
})
# hyperparameters from https://github.com/clinc/oos-eval/blob/master/hyperparameters.csv
learning_rate = 4.00e-05
warmup_proportion = 0.1
train_batch_size = 32
eval_batch_size = 32
num_train_epochs = 5
gradient_accumulation_steps = 1

huggingface_modelname = "bert-large-uncased"

In [20]:
id2label = {i: name for i, name in enumerate(dataset["train"].features["labels"].names)}
label2id = {name: i for i, name in id2label.items()}
oos_label = label2id['oos']

In [21]:
oos_label

42

In [22]:
(np.array(dataset["test"]["labels"]) == oos_label).sum()

240

In [23]:
dataset["train"] = dataset["train"].filter(lambda x: x['labels'] != oos_label)
dataset["validation"] = dataset["validation"].filter(lambda x: x['labels'] != oos_label)
# dataset["test"] = dataset["test"]

Filter:   0%|          | 0/11535 [00:00<?, ? examples/s]

Filter:   0%|          | 0/3845 [00:00<?, ? examples/s]

In [24]:
len(dataset['train'].features['labels'].names)-1

150

In [25]:
dataset['train'].features['labels'].names[42]

'oos'

In [26]:
torch.mps.current_allocated_memory()

0

In [27]:
tokenizer = AutoTokenizer.from_pretrained(huggingface_modelname)

def tokenize_function(examples):
    return tokenizer(examples["text"], truncation=True)

tokenized_datasets = dataset.map(tokenize_function, batched=True)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

assert dataset["train"].features["labels"].names == dataset["test"].features["labels"].names 

Map:   0%|          | 0/10815 [00:00<?, ? examples/s]

Map:   0%|          | 0/3845 [00:00<?, ? examples/s]

Map:   0%|          | 0/3605 [00:00<?, ? examples/s]

In [28]:
model = AutoModelForSequenceClassification.from_pretrained(
    huggingface_modelname,
    num_labels=len(id2label),
    id2label=id2label,
    label2id=label2id
    )

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-large-uncased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [29]:
torch.mps.current_allocated_memory()

0

In [30]:
training_args = TrainingArguments(
    output_dir="test_trainer",
    learning_rate=learning_rate,
    num_train_epochs=num_train_epochs,
    per_device_train_batch_size=train_batch_size,
    per_device_eval_batch_size=eval_batch_size,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    gradient_accumulation_steps=gradient_accumulation_steps,
    load_best_model_at_end=True,
    )

In [31]:
metric = evaluate.load("accuracy")

In [32]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [33]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

In [34]:
trainer.train()

  0%|          | 0/1690 [00:00<?, ?it/s]

You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


  0%|          | 0/113 [00:00<?, ?it/s]

{'eval_loss': 0.6858530044555664, 'eval_accuracy': 0.9256588072122053, 'eval_runtime': 29.9899, 'eval_samples_per_second': 120.207, 'eval_steps_per_second': 3.768, 'epoch': 1.0}
{'loss': 2.2109, 'learning_rate': 2.8165680473372784e-05, 'epoch': 1.48}


  0%|          | 0/113 [00:00<?, ?it/s]

...

  0%|          | 0/113 [00:00<?, ?it/s]

{'eval_loss': 0.10274676978588104, 'eval_accuracy': 0.9766990291262136, 'eval_runtime': 28.5654, 'eval_samples_per_second': 126.202, 'eval_steps_per_second': 3.956, 'epoch': 4.0}
{'loss': 0.0266, 'learning_rate': 4.497041420118343e-06, 'epoch': 4.44}


  0%|          | 0/113 [00:00<?, ?it/s]

{'eval_loss': 0.10200528055429459, 'eval_accuracy': 0.9772538141470181, 'eval_runtime': 28.2144, 'eval_samples_per_second': 127.772, 'eval_steps_per_second': 4.005, 'epoch': 5.0}
{'train_runtime': 2072.5499, 'train_samples_per_second': 26.091, 'train_steps_per_second': 0.815, 'train_loss': 0.704913727630525, 'epoch': 5.0}


TrainOutput(global_step=1690, training_loss=0.704913727630525, metrics={'train_runtime': 2072.5499, 'train_samples_per_second': 26.091, 'train_steps_per_second': 0.815, 'train_loss': 0.704913727630525, 'epoch': 5.0})

In [35]:
pred_train = trainer.predict(tokenized_datasets["train"])
pred_val = trainer.predict(tokenized_datasets["validation"])
pred_test = trainer.predict(tokenized_datasets["test"])

  0%|          | 0/338 [00:00<?, ?it/s]

  0%|          | 0/113 [00:00<?, ?it/s]

  0%|          | 0/121 [00:00<?, ?it/s]

In [36]:
train_sm = scipy.special.softmax(pred_train.predictions, axis=1)
val_sm = scipy.special.softmax(pred_val.predictions, axis=1)
test_sm = scipy.special.softmax(pred_test.predictions, axis=1)

np.testing.assert_allclose(train_sm.sum(axis=1), 1.0, rtol=1e-6)
np.testing.assert_allclose(val_sm.sum(axis=1), 1.0, rtol=1e-6)
np.testing.assert_allclose(test_sm.sum(axis=1), 1.0, rtol=1e-6)

In [37]:
train_sm_dict = {
    f'label_{i}':  train_sm[:,i] for i in range(train_sm.shape[1])
}
val_sm_dict = {
    f'label_{i}':  val_sm[:,i] for i in range(val_sm.shape[1])
}
test_sm_dict = {
    f'label_{i}':  test_sm[:,i] for i in range(test_sm.shape[1])
}
train_frame = pd.DataFrame(
    {"text": dataset["train"]["text"],
     "hash": range(len(dataset["train"]["text"])),
     "label": dataset["train"]["labels"],
     **train_sm_dict
    }
)
val_frame = pd.DataFrame(
    {"text": dataset["validation"]["text"],
     "hash": range(len(dataset["validation"]["text"])),
     "label": dataset["validation"]["labels"],
     **val_sm_dict
    }
)
test_frame = pd.DataFrame(
    {"text": dataset["test"]["text"],
     "hash": range(len(dataset["test"]["text"])),
     "label": dataset["test"]["labels"],
     **test_sm_dict
    }
)

In [38]:
train_frame.to_csv('data/clinc150/bert/train_predictions.csv', index=False)
val_frame.to_csv('data/clinc150/bert/calibration_predictions.csv', index=False)
test_frame.to_csv('data/clinc150/bert/test_predictions.csv', index=False)

pd.DataFrame({'0': id2label.values()}).to_csv('data/clinc150/bert/labels.csv')

In [57]:
dataset_ood = load_dataset("clinc_oos", "imbalanced").rename_column("intent", "labels")
dataset_ood_full = concatenate_datasets([dataset_ood["train"], dataset_ood["test"], dataset_ood['validation']]).shuffle(seed=42)
# dataset_ood = dataset_ood_full.filter(lambda x: x.labels == 'oos')
dataset_ood_full = dataset_ood_full.filter(lambda x: x['labels'] == label2id['oos'])
dataset_ood_full['text']

tokenized_ood = dataset_ood_full.map(tokenize_function, batched=True)

pred_ood = trainer.predict(tokenized_ood)
ood_sm = scipy.special.softmax(pred_ood.predictions, axis=1)
ood_sm_dict = {
    f'label_{i}':  ood_sm[:,i] for i in range(ood_sm.shape[1])
}
ood_frame = pd.DataFrame(
    {"text": dataset_ood_full["text"],
     "hash": range(len(dataset_ood_full["text"])),
     "label": dataset_ood_full["labels"],
     **ood_sm_dict
    }
)
ood_frame.to_csv('data/clinc150/bert/ood_predictions.csv', index=False)


  0%|          | 0/38 [00:00<?, ?it/s]